In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.autograd import Variable

from PIL import Image
import glob,os
from imageio import imread

import sys

Use the GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
torch.set_default_tensor_type('torch.cuda.FloatTensor')

## Load the data and Visualize

In [ ]:
data_dir ="./commoncharacter>1250/"
os.listdir(data_dir)[0]
names = []
for j in os.listdir(data_dir):
#     print(j, len( glob.glob(data_dir+j+"/*.png")))
    file = glob.glob(data_dir+j)        
    names.append(file)

In [ ]:
for i in data_dir+j+"/*.png":
    file = glob.glob(data_dir+j+"/*.png")[0]
    print(file)
    img = imread(file)
    plt.imshow(img)
    plt.show()
    sys.exit()

Separate the data into a training set and a test set.

In [1]:
def load_split_train_test(datadir, valid_size = .2):
    train_transforms = transforms.Compose([transforms.Resize(size_IMG),
                                       transforms.ToTensor(),
                                       ])
    
    test_transforms = transforms.Compose([transforms.Resize(size_IMG),
                                      transforms.ToTensor(),
                                      ])
    
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    
    num_train = len(train_data)
    print("number of training data is", num_train)
    
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    
    np.random.shuffle(indices)
    
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size)
    
    return trainloader, testloader


## Use a CNN to train machine to learn chinese

The CNN we use is very simple. There are two Convolutional layers for feature extraction followed by two fully connected layers that perform classification.

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(8*8 * 64, 100)
        self.fc2 = nn.Linear(100, num_classes)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

Define training parameters

In [ ]:
train_loader, test_loader = load_split_train_test(data_dir, .2)
print(train_loader.dataset.classes)

num_epochs = 5
num_classes = len(os.listdir(data_dir))
print("num of classes", num_classes)
batch_size = 100
learning_rate = 0.001
size_IMG = 32

Define model, optimizer and loss function. We use the ADAM optimizer and a Cross Entropy Loss

In [ ]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.to(device)

Training the model

In [ ]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        single_channel_image = images[:,0,:,:].view(images.size()[0],1,size_IMG,size_IMG).cuda()
        outputs = model(single_channel_image)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

## Plot results & save model

In [ ]:
plt.plot(loss_list)
plt.plot(acc_list)

In [ ]:
print(model.state_dict())

In [ ]:
torch.save(model.state_dict(), "./conv_network_dic")
torch.save(model, "./conv_network_model")


In [ ]:
np.save("./labels.npy", train_loader.dataset.classes)